In [15]:
%pip install requests feedparser bs4 pandas

  Using cached pandas-2.3.0-cp313-cp313-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.3.0-cp313-cp313-win_amd64.whl (11.0 MB)
   ---------------------------------------- 0.0/12.7 MB ? eta -:--:--
    --------------------------------------- 0.3/12.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.7 MB 1.5 MB/s eta 0:00:09
   - -------------------------------------- 0.5/12.7 MB 1.5 MB/s eta 0:00:09
   -- ------------------------------------- 0.8/12.7 MB 986.9 kB/s eta 0:00:13
   --- ------------------------------------ 1.0/12.7 MB 1.0 MB/s eta 0:00:12
   ---- ----------------------------------- 1.6/12.7 MB 1.3 MB/s eta 0:00:09
   ----- ---------------------------------- 1.8/12.7 MB 1.4 MB/s eta 0:00:08
   ----- ---------------------------------- 1.8/12.7 MB 1.4 MB/s eta 0:00:08
   ----- ---------------------------------- 1.8/12.7 

In [6]:
import requests
import feedparser
import time

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://www.google.com/",
}

rss_urls = [
    'https://www.tribunnews.com/rss',
    'https://medan.tribunnews.com/rss',
    'https://jabar.tribunnews.com/rss',
    'https://surabaya.tribunnews.com/rss',
    'http://jabar.tribunnews.com/rss',
    'https://wartakota.tribunnews.com/rss',
    'https://jogja.tribunnews.com/rss',
    'http://jateng.tribunnews.com/rss',
    'http://aceh.tribunnews.com/rss',
]

urls = set()
for rss_url in rss_urls:
    try:
        print(f"Mengambil dari: {rss_url}")
        resp = requests.get(rss_url, headers=headers, timeout=10)
        feed = feedparser.parse(resp.text)
        for entry in feed.entries:
            urls.add(entry.link)
        time.sleep(1)
    except Exception as e:
        print(f"❌ Gagal ambil dari {rss_url}: {e}")

urls = list(urls)[:1000]
print(f"✅ Total link yang berhasil dikumpulkan: {len(urls)}")


Mengambil dari: https://www.tribunnews.com/rss
Mengambil dari: https://medan.tribunnews.com/rss
Mengambil dari: https://jabar.tribunnews.com/rss
Mengambil dari: https://surabaya.tribunnews.com/rss
Mengambil dari: http://jabar.tribunnews.com/rss
Mengambil dari: https://wartakota.tribunnews.com/rss
Mengambil dari: https://jogja.tribunnews.com/rss
Mengambil dari: http://jateng.tribunnews.com/rss
Mengambil dari: http://aceh.tribunnews.com/rss
✅ Total link yang berhasil dikumpulkan: 160


In [9]:
import requests
from bs4 import BeautifulSoup

def scrape_berita(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36",
        "Referer": "https://www.google.com/",
        "Accept-Language": "id,en-US;q=0.9,en;q=0.8",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    }
    try:
        resp = requests.get(url, headers=headers, timeout=10)
        if resp.status_code != 200:
            print(f"  ⚠️ Status code: {resp.status_code}")
            return None
        soup = BeautifulSoup(resp.content, "html.parser")

        judul_tag = soup.select_one("h1")
        if not judul_tag:
            print("  ⚠️ Tidak menemukan tag <h1>")
            return None

        isi_paragraf = soup.select(".side-article p")
        if not isi_paragraf:
            print("  ⚠️ Tidak menemukan isi artikel")
            return None

        judul = judul_tag.text.strip()
        isi = "\n".join([p.text.strip() for p in isi_paragraf if p.text.strip()])

        tanggal_tag = soup.select_one("time") or soup.select_one(".date")
        tanggal = tanggal_tag.text.strip() if tanggal_tag else "Tidak ditemukan"

        breadcrumb = soup.select(".breadcrumb li a")
        kategori = breadcrumb[1].text.strip() if len(breadcrumb) > 1 else "Tidak diketahui"

        return {
            "Sumber": "Tribunnews",
            "Judul": judul,
            "Isi": isi,
            "Tanggal": tanggal,
            "Kategori": kategori,
            "Link": url,
        }

    except Exception as e:
        print(f"  ❌ Gagal scraping {url}: {e}")
        return None


In [10]:
data = scrape_berita(urls[0])  # Ganti dengan URL yang ingin diuji

# print hasil scraping
print(data)

{'Sumber': 'Tribunnews', 'Judul': 'Jutaan Jamaah Haji Tinggalkan Mina, Persiapan Tawaf Al-Wida Berjalan Lancar', 'Isi': "TRIBUNNEWS.COM - Sebagian besar jamaah Haji telah meninggalkan Mina setelah menjalankan ritual lempar jumrah pada hari kedua Tashreeq, Minggu (8/6/2025) sore.\nRitual ini menjadi salah satu bagian penting dari ibadah Haji.\nDi mana jamaah melempar kerikil ke tiga pilar Jamarat yang melambangkan penolakan terhadap godaan setan, Arab News melaporkan.\nJamaah yang meninggalkan Mina sebelum matahari terbenam hari kedua Tashreeq diperbolehkan kembali ke Makkah untuk melanjutkan rangkaian ibadah, termasuk Tawaf Al-Wida.\nSedangkan jamaah yang belum meninggalkan Mina diwajibkan menginap satu hari lagi dan mengulangi ritual lempar jumrah pada hari ketiga Tashreeq, yaitu Senin (9/6/2025).\nSetelah menyelesaikan ritual lempar jumrah, jamaah yang bergegas langsung menuju Masjidil Haram untuk melaksanakan Tawaf Al-Wida.\nSebagai catatan, Tawaf Al-Wida yaitu putaran terakhir meng

In [11]:
berita_list = []
for i, url in enumerate(urls):
    print(f"[{i+1}/{len(urls)}] Scraping: {url}")
    data = scrape_berita(url)
    print("  ↳ Data berhasil?" , bool(data))
    if data:
        berita_list.append(data)
    time.sleep(1)

df = pd.DataFrame(berita_list)
df.to_csv("berita_tribunnews_lengkap.csv", index=False)
print("✅ CSV disimpan! Jumlah artikel:", len(df))


[1/160] Scraping: https://www.tribunnews.com/internasional/2025/06/09/jutaan-jamaah-haji-tinggalkan-mina-persiapan-tawaf-al-wida-berjalan-lancar
  ↳ Data berhasil? True
[2/160] Scraping: https://medan.tribunnews.com/2025/06/09/satlantas-polres-batubara-sosialisasikan-tertib-berlalu-lintas-di-kecamatan-sei-suka-dan-air-putih
  ↳ Data berhasil? True
[3/160] Scraping: https://wartakota.tribunnews.com/2025/06/09/para-rider-uji-langsung-ban-michelin-power-gp2-dan-cup2-di-sirkuit-mandalika-ini-hasilnya
  ↳ Data berhasil? True
[4/160] Scraping: https://jateng.tribunnews.com/2025/06/09/7-fakta-rangkuman-skandal-fajar-shabrina-indonesia-idol-dan-amanda-lucson-cinta-segitiga
  ↳ Data berhasil? True
[5/160] Scraping: https://aceh.tribunnews.com/2025/06/09/penyelidikan-di-kek-arun-besok-kejari-lhokseumawe-mulai-mintai-keterangan-pt-patna-pag-dan-pim
  ↳ Data berhasil? True
[6/160] Scraping: https://jateng.tribunnews.com/2025/06/09/kuatkan-sinergi-pelaku-wisata-dan-pemda-bupati-fahmi-road-trip-ke-b

KeyboardInterrupt: 

In [ ]:
from google.colab import files
files.download("berita_tribunnews_lengkap.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>